In [29]:
import pandas as pd
import numpy as np

In [31]:
wd = '/Users/kevin/Dropbox/Policy Experimentation/'

# full panel:
df = pd.read_excel(wd+'R&R/incentives/Pref_Secretary&Mayor_2021nov.xlsx')
df = df[df.pos == 1]
df['off_yr'] = df.apply(lambda x: 2022 if x['incumbent'] == 1 else x['off_yr'], axis=1)
df.drop_duplicates(subset=['name', 'prefname'], inplace=True)
df['on_yr'] = df.apply(lambda x: x['on_yr']-1 if x['on_mh']<=6 else x['on_yr'], axis=1)
df['off_yr'] = df.apply(lambda x: x['off_yr']-1 if x['off_mh']<=6 else x['off_yr'], axis=1)

# Create a panel data at prefecture-year level
idx = pd.MultiIndex.from_product(
    [df.prefname.unique(), range(1997, 2022)], names=['prefname', 'year']
)
df_new = pd.DataFrame(index=idx).reset_index()

def find_mayor(data, prefname, year):
    mayor = data[(data.prefname == prefname) & (data.on_yr <= year) & (data.off_yr > year)]['name'].unique()
    if len(mayor)>=1:
        return mayor[0]
    else:
        return 'NotFound'

df_new['name_party'] = df_new.apply(lambda x: find_mayor(df, x['prefname'], x['year']), axis=1)
df_new['pos']= 1
df_new = pd.merge(df_new, df[['name', 'prefname', 'plevel_begin', 'on_yr', 'on_mh', 'off_yr', 'off_mh', 'center', 'otherprovbefore', 'birth_yr', 'promotion', 'birth_mh', 'pos']].rename({
    'name':'name_party'
}, axis=1), on=['name_party', 'prefname', 'pos'], how='left')
df_new['birth_yr'] = df_new.apply(lambda x: x['birth_yr'] +1 if x['birth_mh'] <= 6 else x['birth_yr']+2, axis=1)
df_new['age'] = df_new.year - df_new.birth_yr

pref_to_code = dict(df[['prefname', 'prefecture']].values)
df_new['prefecture'] = df_new.prefname.apply(lambda x: pref_to_code[x])

wd = '/Users/kevin/Dropbox/legal_censorship/'
df_new.to_excel(wd+'Data/career_incentive_panel.xlsx', index=False)

# (df_new.name_party == "NotFound").mean() == 0.03

In [32]:
df_new

,prefname,year,name_party,pos,plevel_begin,on_yr,on_mh,off_yr,off_mh,center,otherprovbefore,birth_yr,promotion,birth_mh,age,prefecture
0,石家庄市,1997,陈来立,1,副部,1997.0,12.0,2000.0,6.0,0.0,0.0,1945.0,1.0,NaN,52.0,130100.0
1,石家庄市,1998,陈来立,1,副部,1997.0,12.0,2000.0,6.0,0.0,0.0,1945.0,1.0,NaN,53.0,130100.0
2,石家庄市,1999,陈来立,1,副部,1997.0,12.0,2000.0,6.0,0.0,0.0,1945.0,1.0,NaN,54.0,130100.0
3,石家庄市,2000,吴振华,1,副部,2000.0,6.0,2006.0,9.0,0.0,0.0,1948.0,1.0,7.0,52.0,130100.0
4,石家庄市,2001,吴振华,1,副部,2000.0,6.0,2006.0,9.0,0.0,0.0,1948.0,1.0,7.0,53.0,130100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8370,阿勒泰地区,2017,张岩,1,正厅,2016.0,3.0,2022.0,NaN,0.0,0.0,1965.0,NaN,2.0,52.0,654300.0
8371,阿勒泰地区,2018,张岩,1,正厅,2016.0,3.0,2022.0,NaN,0.0,0.0,1965.0,NaN,2.0,53.0,654300.0
8372,阿勒泰地区,2019,张岩,1,正厅,2016.0,3.0,2022.0,NaN,0.0,0.0,1965.0,NaN,2.0,54.0,654300.0
8373,阿勒泰地区,2020,张岩,1,正厅,2016.0,3.0,2022.0,NaN,0.0,0.0,1965.0,NaN,2.0,55.0,654300.0
